In [ ]:
from src.common import config

## 1. Scraping

In [ ]:
import src.scraping as scr

In [ ]:
_ = await scr.scrape_ids()

In [ ]:
await scr.download_docs()

In [ ]:
_ = scr.extract_text()

In [ ]:
_ = scr.parse_docs()

## 2. Labeling

In [ ]:
from src.labeling import label_docs

In [ ]:
_ = await label_docs()

## 3. Augmentation

In [ ]:
from src.augmentation import create_augmentations

In [ ]:
_ = await create_augmentations()

## 4. Train and Test Sets

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

In [ ]:
df = pd.read_json(config.DOCS_AUGMENTED_JSONL, lines=True).sort_values(by="id")

In [ ]:
train_unbalanced, test = train_test_split(
    df,
    test_size=1/3,
    stratify=df.decision,
    random_state=42,
    shuffle=True
)

In [ ]:
n = train_unbalanced.decision.value_counts().min()
train = (
    train_unbalanced.groupby("decision")
    .sample(n=n, random_state=42)
    .sample(frac=1, random_state=42)
)

In [ ]:
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

In [ ]:
train_dataset = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)

dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

In [ ]:
dataset.save_to_disk(
    config.DATA_DIR / "BGH-CivAppeals-GenderCF"
)